In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from xgboost import XGBRegressor
import copy

In [2]:
data = pd.read_csv('data.csv')
data.head()

,Datetime,temp,templow,icon,baro,hum,rain,fog,thunder,snow,...,desc_Light rain. Passing clouds.,desc_Low clouds.,desc_More clouds than sun.,desc_Mostly cloudy.,desc_Overcast.,desc_Partly cloudy.,desc_Partly sunny.,desc_Passing clouds.,desc_Scattered clouds.,desc_Sunny.
0,2019-10-01 00:00:00,2.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-10-01 01:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-10-01 02:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-10-01 03:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-10-01 04:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
drop = ['etat_barre_ce', 'etat_barre_lc', 'etat_barre_pv', 'Année', 'Mois', 'Jour', 'Heure', 'Jour semaine']
target_ce = ['q_ce', 'k_ce']
target_lc = ['q_lc', 'k_lc']
target_pv = ['q_pv', 'k_pv']
all_ = drop + target_ce + target_lc + target_pv
features = [x for x in data.columns.tolist() if x not in all_]
df = copy.deepcopy(data)
df = df.drop(drop, axis=1)
df_ce = copy.deepcopy(df[features + target_ce])
df_lc = copy.deepcopy(df[features + target_lc])
df_pv = copy.deepcopy(df[features + target_pv])

In [9]:
df_ce.head()

,Datetime,temp,templow,icon,baro,hum,rain,fog,thunder,snow,...,desc_More clouds than sun.,desc_Mostly cloudy.,desc_Overcast.,desc_Partly cloudy.,desc_Partly sunny.,desc_Passing clouds.,desc_Scattered clouds.,desc_Sunny.,q_ce,k_ce
0,2019-10-01 00:00:00,2.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,767.0,10.40890
1,2019-10-01 01:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,695.0,8.73556
2,2019-10-01 02:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,423.0,5.25167
3,2019-10-01 03:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.0,3.79667
4,2019-10-01 04:00:00,1.0,1.0,13.0,1026.0,88.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,331.0,3.73000


In [43]:
x_.shape

(120, 102)

# Bazooka 1 : Stacking models

## Champs Elysées

### Q

In [34]:
train = df_ce[(df_ce.Datetime < '2020-11-25') & (df_ce.q_ce.notnull())]
test = df_ce[(df_ce.Datetime >= '2020-11-25') & (df_ce.Datetime < '2020-11-30') & (df_ce.q_ce.notnull())]
X_train = train.drop(['Datetime', 'q_ce', 'k_ce'], axis=1)
y_train = train['q_ce']
x_test = test.drop(['Datetime', 'q_ce', 'k_ce'], axis=1)
y_test = test['q_ce']

In [25]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75, learning_rate=0.01, loss="ls", max_depth=6, max_features=0.9500000000000001, min_samples_leaf=4, min_samples_split=20, n_estimators=1000, subsample=0.35000000000000003,random_state=27)),
    XGBRegressor(learning_rate=0.1, max_depth=10, min_child_weight=6, n_estimators=1000, nthread=1, objective="reg:squarederror", subsample=1.0, random_state=27)
    #LGBMRegressor(learning_rate=0.1, max_depth=15, min_child_weight=6, n_estimators=500, nthread=1, objective="reg:squarederror", subsample=1.0, random_state=27)
)
exported_pipeline.fit(X_train, y_train)

Pipeline(steps=[('stackingestimator',
                 StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75,
                                                                       learning_rate=0.01,
                                                                       max_depth=6,
                                                                       max_features=0.9500000000000001,
                                                                       min_samples_leaf=4,
                                                                       min_samples_split=20,
                                                                       n_estimators=1000,
                                                                       random_state=27,
                                                                       subsample=0.35000000000000003))),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsam

In [17]:
from sklearn.metrics import mean_squared_error

In [26]:
np.sqrt(mean_squared_error(exported_pipeline.predict(x_test), y_test.values))

182.59672579824058

In [39]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=LGBMRegressor(learning_rate=0.1, min_child_weight=6, n_estimators=1000, nthread=1, subsample=1.0, random_state=27)),
    XGBRegressor(learning_rate=0.1, max_depth=10, min_child_weight=6, n_estimators=1000, nthread=1, objective="reg:squarederror", subsample=1.0, random_state=27)
)
exported_pipeline.fit(X_train, y_train)

[LightGBM] [Warning] num_threads is set with nthread=1, will be overridden by n_jobs=-1. Current value: num_threads=-1


Pipeline(steps=[('stackingestimator',
                 StackingEstimator(estimator=LGBMRegressor(min_child_weight=6,
                                                           n_estimators=1000,
                                                           nthread=1,
                                                           random_state=27))),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsample_bylevel=1, colsample_bynode=1,
                              colsample_bytree=1, gamma=0, gpu_id=-1,
                              importance_type='gain',
                              interaction_constraints='', learning_rate=0.1,
                              max_delta_step=0, max_depth=10,
                              min_child_weight=6, missing=nan,
                              monotone_constraints='()', n_estimators=1000,
                              n_jobs=1, nthread=1, num_parallel_tree=1,
              

In [42]:
np.sqrt(mean_squared_error(exported_pipeline.predict(X_train), y_train.values))

3.3451036454729075

### K

In [27]:
train = df_ce[(df_ce.Datetime < '2020-11-25') & (df_ce.k_ce.notnull())]
test = df_ce[(df_ce.Datetime >= '2020-11-25') & (df_ce.Datetime < '2020-11-30') & (df_ce.k_ce.notnull())]
X_train = train.drop(['Datetime', 'q_ce', 'k_ce'], axis=1)
y_train = train['k_ce']
x_test = test.drop(['Datetime', 'q_ce', 'k_ce'], axis=1)
y_test = test['k_ce']

In [28]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75, learning_rate=0.01, loss="ls", max_depth=6, max_features=0.9500000000000001, min_samples_leaf=4, min_samples_split=20, n_estimators=1000, subsample=0.35000000000000003,random_state=27)),
    XGBRegressor(learning_rate=0.1, max_depth=10, min_child_weight=6, n_estimators=1000, nthread=1, objective="reg:squarederror", subsample=1.0, random_state=27)
    #LGBMRegressor(learning_rate=0.1, max_depth=15, min_child_weight=6, n_estimators=500, nthread=1, objective="reg:squarederror", subsample=1.0, random_state=27)
)
exported_pipeline.fit(X_train, y_train)

Pipeline(steps=[('stackingestimator',
                 StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.75,
                                                                       learning_rate=0.01,
                                                                       max_depth=6,
                                                                       max_features=0.9500000000000001,
                                                                       min_samples_leaf=4,
                                                                       min_samples_split=20,
                                                                       n_estimators=1000,
                                                                       random_state=27,
                                                                       subsample=0.35000000000000003))),
                ('xgbregressor',
                 XGBRegressor(base_score=0.5, booster='gbtree',
                              colsam

In [29]:
np.sqrt(mean_squared_error(exported_pipeline.predict(x_test), y_test.values))

5.731906332357755

In [32]:
exported_pipeline.predict(x_test)

array([ 2.9802382,  2.8815336,  1.5518559,  2.0161645,  1.585243 ,
        1.0443733,  1.5952708,  3.5895648,  8.923735 , 15.711721 ,
       14.5383835, 16.632332 , 16.208008 , 16.273758 , 15.680016 ,
       17.081106 , 16.250599 , 15.712782 ,  9.820754 ,  9.323345 ,
       12.429255 , 10.476629 ,  9.134832 ,  4.6016116,  5.5057244,
        3.477779 ,  2.7201803,  1.7320745,  1.2067641,  0.9482653,
        1.9738827,  4.2479086,  8.049665 , 14.87104  , 18.196215 ,
       17.489155 , 16.33226  , 16.010637 , 17.247385 , 17.543629 ,
       16.000517 , 16.740118 , 15.820739 , 15.974157 , 12.9145775,
       12.057243 , 10.329183 ,  6.0079823,  5.4684486,  4.473836 ,
        3.714355 ,  2.7846088,  2.1554704,  1.2896713,  1.2986009,
        3.2124333,  6.842898 , 11.151227 , 17.076004 , 15.436073 ,
       15.703313 , 16.700127 , 18.022966 , 16.559444 , 15.770129 ,
       15.422131 , 16.097622 , 11.338148 , 12.401517 , 11.125065 ,
       10.342988 ,  6.6655755, 10.428204 , 10.012559 ,  8.1068

In [33]:
y_test.values

array([ 2.74167,  1.57833,  1.60723,  1.29389,  0.755  ,  0.75722,
        0.95667,  1.79778,  5.44834, 10.79389, 16.31667, 20.77056,
       13.50611, 10.01389,  9.85278, 10.10445,  9.94778,  9.83778,
       10.34556,  9.63278, 10.56278,  6.75833,  4.775  ,  3.25334,
        2.17667,  1.71334,  1.26445,  0.91556,  0.66278,  0.62667,
        0.77444,  1.875  ,  5.63778, 10.60389, 12.93445, 14.08834,
       12.00945, 11.29945, 10.00167,  9.24667, 11.71389, 10.57167,
       11.31278, 10.07889, 10.18667,  8.23167,  4.94944,  3.53889,
        2.19445,  1.73722,  1.545  ,  1.43556,  0.87667,  0.71944,
        1.16611,  1.59556,  6.34611, 12.42778, 13.39778, 13.88222,
       13.10667, 10.30167, 10.85389, 12.72111, 10.35833, 12.31056,
       11.765  , 10.49833,  9.85556,  8.21889,  5.46834,  5.67   ,
        2.46667,  1.93055,  1.99667,  1.42334,  0.99278,  0.51833,
        0.61056,  1.26222,  2.71167,  3.25889,  4.39222,  7.34556,
        9.42278, 11.37111, 12.14667, 11.07167, 14.03389, 24.91

https://github.com/xiaochus/TrafficFlowPrediction